## Actividad:  Redes neuronales recurrentes como compartición de pesos

Las redes neuronales recurrentes (RNNs) son una clase de redes neuronales que son especialmente adecuadas para procesar datos secuenciales o temporales, como texto, audio o series temporales. Una característica clave de las RNNs es la **compartición de pesos**, lo que permite a la red generalizar mejor en secuencias de diferentes longitudes y capturar dependencias temporales.

#### Compartición de pesos en una red totalmente conectada

En una red neuronal totalmente conectada tradicional, cada capa tiene su propio conjunto de pesos y sesgos. Si consideramos una red con $L$ capas, la salida de la capa $l$ se puede expresar como:

$$
\mathbf{h}^{(l)} = \sigma(\mathbf{W}^{(l)} \mathbf{h}^{(l-1)} + \mathbf{b}^{(l)})
$$

Donde:

- $\mathbf{W}^{(l)}$ es la matriz de pesos de la capa $l$.
- $\mathbf{b}^{(l)}$ es el vector de sesgos.
- $\mathbf{h}^{(l-1)}$ es la salida de la capa anterior.
- $\sigma$ es una función de activación no lineal.

En este enfoque, cada capa aprende sus propios pesos, lo que puede conducir a un gran número de parámetros y potencialmente al sobreajuste, especialmente si el conjunto de datos es pequeño.

#### Compartición de pesos en el tiempo

En las RNNs, la idea clave es reutilizar el mismo conjunto de pesos en cada paso temporal. Esto significa que para cada paso de tiempo $t$, la actualización del estado oculto $\mathbf{h}_t$ se realiza utilizando los mismos pesos $\mathbf{W}$ y $\mathbf{U}$:

$$
\mathbf{h}_t = \sigma(\mathbf{W} \mathbf{x}_t + \mathbf{U} \mathbf{h}_{t-1} + \mathbf{b})
$$

Donde:

- $\mathbf{x}_t$ es la entrada en el tiempo $t$.
- $\mathbf{h}_{t-1}$ es el estado oculto anterior.
- $\mathbf{W}$ es la matriz de pesos de entrada a oculto.
- $\mathbf{U}$ es la matriz de pesos oculto a oculto (recurrente).
- $\mathbf{b}$ es el vector de sesgos.

La compartición de pesos permite a la red aplicar la misma transformación en cada paso temporal, lo que es esencial para capturar patrones secuenciales y reducir el número total de parámetros.

### RNNs en PyTorch

PyTorch es un framework popular para el aprendizaje profundo que ofrece una implementación eficiente y flexible de RNNs. A continuación, exploraremos cómo construir y entrenar RNNs en PyTorch para resolver un problema sencillo de clasificación de secuencias.

#### Un problema simple de clasificación de secuencias

Supongamos que queremos clasificar secuencias de números enteros en dos categorías:

- **Clase 0**: La suma de los números es par.
- **Clase 1**: La suma de los números es impar.

El objetivo es construir una RNN que pueda procesar secuencias de longitud variable y predecir la categoría correcta.

#### Generación de datos

Para entrenar el modelo, necesitamos generar secuencias aleatorias y sus etiquetas correspondientes.

```python
import torch
import random

def generate_sequence(max_length=10):
    length = random.randint(1, max_length)
    sequence = [random.randint(0, 9) for _ in range(length)]
    label = sum(sequence) % 2  # 0 si la suma es par, 1 si es impar
    return sequence, label
```

#### Capas de embedding

Antes de alimentar los números enteros a la RNN, es común transformarlos en vectores de embedding. Una capa de embedding aprende una representación densa de los índices enteros.

En PyTorch, podemos definir una capa de embedding:

```python
import torch.nn as nn

embedding_dim = 16
vocab_size = 10  # Los dígitos del 0 al 9
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
```

Esto convierte cada entero en un vector de dimensión `embedding_dim`.



### Definición de la RNN

Definimos una RNN simple utilizando `nn.RNN`:

```python
hidden_size = 32
rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
```

- `input_size` es la dimensión del embedding.
- `hidden_size` es la dimensión del estado oculto.


Después de procesar la secuencia completa con la RNN, utilizamos el estado oculto del último paso temporal para hacer la predicción:

```python
output_dim = 2  # Dos clases: par e impar
output_layer = nn.Linear(hidden_size, output_dim)
```

El flujo completo es:

1. Pasar la secuencia a través de la capa de embedding.
2. Procesar los embeddings con la RNN.
3. Extraer el estado oculto del último tiempo.
4. Pasar el estado oculto por la capa lineal para obtener las predicciones.

#### Ejemplo del paso hacía adelante

```python
def forward_pass(sequence):
    embedded = embedding(sequence)  # [batch_size, seq_length, embedding_dim]
    output, hidden = rnn(embedded)  # hidden: [1, batch_size, hidden_size]
    logits = output_layer(hidden.squeeze(0))  # [batch_size, output_dim]
    return logits
```

### Mejorando el tiempo de entrenamiento con packing

Cuando trabajamos con secuencias de diferentes longitudes, es eficiente agruparlas en lotes y procesarlas simultáneamente. Sin embargo, necesitamos manejar el padding y packing de secuencias.

#### Pad y  pack

El padding implica rellenar las secuencias más cortas con tokens especiales (por ejemplo, ceros) para igualar la longitud de las secuencias en el lote.

```python
from torch.nn.utils.rnn import pad_sequence

sequences = [torch.tensor(seq) for seq, label in batch]
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
```

Sin embargo, procesar estos tokens de padding puede ser ineficiente.

El packing permite a PyTorch ignorar los pasos de padding durante el procesamiento de la RNN.

```python
from torch.nn.utils.rnn import pack_padded_sequence

lengths = [len(seq) for seq in sequences]
packed_input = pack_padded_sequence(embedded_sequences, lengths, batch_first=True, enforce_sorted=False)
```


La capa de embedding debe aplicarse antes de empaquetar las secuencias, ya que no puede manejar secuencias empaquetadas.

```python
embedded_sequences = embedding(padded_sequences)
packed_input = pack_padded_sequence(embedded_sequences, lengths, batch_first=True, enforce_sorted=False)
```


Al entrenar la RNN en lotes, mejoramos significativamente la eficiencia computacional.

```python
output_packed, hidden = rnn(packed_input)
```


En algunos casos, es necesario manejar tanto secuencias empaquetadas como no empaquetadas. Por ejemplo, cuando combinamos datos que requieren diferentes formas de procesamiento. Es importante asegurar que las dimensiones y tipos de datos sean compatibles en todo el flujo.



### RNNs más complejas

Las RNNs básicas pueden tener limitaciones en su capacidad para capturar dependencias a largo plazo. Existen arquitecturas más complejas que abordan estas limitaciones.

#### Múltiples capas

Agregar múltiples capas a una RNN puede aumentar su capacidad para modelar patrones complejos. En PyTorch, podemos especificar el número de capas:

```python
num_layers = 2
rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
```

La salida de cada capa se utiliza como entrada para la siguiente capa.

#### RNN bidireccionales

Las RNNs bidireccionales procesan la secuencia en ambas direcciones: hacia adelante y hacia atrás. Esto permite a la red tener en cuenta tanto el contexto pasado como el futuro en cada punto temporal.

En PyTorch:

```python
rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, bidirectional=True, batch_first=True)
```

La salida tendrá el doble de dimensión, ya que se concatenan los estados ocultos de ambas direcciones.

#### Ecuaciones para una  RNN bidireccional

Para la dirección hacia adelante:

$$
\overrightarrow{\mathbf{h}}_t = \sigma(\mathbf{W} \mathbf{x}_t + \mathbf{U} \overrightarrow{\mathbf{h}}_{t-1} + \mathbf{b})
$$

Para la dirección hacia atrás:

$$
\overleftarrow{\mathbf{h}}_t = \sigma(\mathbf{W} \mathbf{x}_t + \mathbf{U} \overleftarrow{\mathbf{h}}_{t+1} + \mathbf{b})
$$

La salida combinada es:

$$
\mathbf{h}_t = [\overrightarrow{\mathbf{h}}_t; \overleftarrow{\mathbf{h}}_t]
$$

Donde $[;]$ denota la concatenación.

### Long short-term memory (LSTM) y gated recurrent unit (GRU)

Para abordar el problema de los gradientes desvanecientes en RNNs simples, se introdujeron las arquitecturas LSTM y GRU, que utilizan puertas para controlar el flujo de información.

#### LSTM

El LSTM introduce celdas de memoria y puertas de entrada, olvido y salida:

$$
\begin{align*}
\mathbf{i}_t &= \sigma(\mathbf{W}_i \mathbf{x}_t + \mathbf{U}_i \mathbf{h}_{t-1} + \mathbf{b}_i) \\
\mathbf{f}_t &= \sigma(\mathbf{W}_f \mathbf{x}_t + \mathbf{U}_f \mathbf{h}_{t-1} + \mathbf{b}_f) \\
\mathbf{o}_t &= \sigma(\mathbf{W}_o \mathbf{x}_t + \mathbf{U}_o \mathbf{h}_{t-1} + \mathbf{b}_o) \\
\mathbf{c}_t &= \mathbf{f}_t \odot \mathbf{c}_{t-1} + \mathbf{i}_t \odot \tanh(\mathbf{W}_c \mathbf{x}_t + \mathbf{U}_c \mathbf{h}_{t-1} + \mathbf{b}_c) \\
\mathbf{h}_t &= \mathbf{o}_t \odot \tanh(\mathbf{c}_t)
\end{align*}
$$

Donde:

- $\mathbf{i}_t$: puerta de entrada.
- $\mathbf{f}_t$: puerta de olvido.
- $\mathbf{o}_t$: puerta de salida.
- $\mathbf{c}_t$: estado de la celda.
- $\odot$: multiplicación elemento a elemento.

#### GRU

El GRU simplifica el LSTM combinando algunas puertas:

$$
\begin{align*}
\mathbf{z}_t &= \sigma(\mathbf{W}_z \mathbf{x}_t + \mathbf{U}_z \mathbf{h}_{t-1} + \mathbf{b}_z) \\
\mathbf{r}_t &= \sigma(\mathbf{W}_r \mathbf{x}_t + \mathbf{U}_r \mathbf{h}_{t-1} + \mathbf{b}_r) \\
\mathbf{n}_t &= \tanh(\mathbf{W}_n \mathbf{x}_t + \mathbf{U}_n (\mathbf{r}_t \odot \mathbf{h}_{t-1}) + \mathbf{b}_n) \\
\mathbf{h}_t &= (1 - \mathbf{z}_t) \odot \mathbf{n}_t + \mathbf{z}_t \odot \mathbf{h}_{t-1}
\end{align*}
$$

#### Implementación en PyTorch

Para utilizar LSTM o GRU, simplemente reemplazamos `nn.RNN` con `nn.LSTM` o `nn.GRU`:

```python
lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
```


### Ejemplo

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import random

# Generación de datos
def generar_secuencia(max_longitud=10):
    longitud = random.randint(1, max_longitud)
    secuencia = [random.randint(0, 9) for _ in range(longitud)]
    etiqueta = sum(secuencia) % 2  # 0 si es par, 1 si es impar
    return secuencia, etiqueta

def generar_lote(tamano_lote, max_longitud=10):
    secuencias = []
    etiquetas = []
    for _ in range(tamano_lote):
        seq, label = generar_secuencia(max_longitud)
        secuencias.append(torch.tensor(seq, dtype=torch.long))
        etiquetas.append(label)
    return secuencias, torch.tensor(etiquetas, dtype=torch.long)

# Definición del modelo
class ClasificadorSecuencia(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_dim, num_layers=2, bidirectional=True):
        super(ClasificadorSecuencia, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.num_directions = 2 if bidirectional else 1
        self.hidden_size = hidden_size

        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True
        )

        self.output_layer = nn.Linear(hidden_size * self.num_directions, output_dim)

    def forward(self, sequences, lengths):
        embedded = self.embedding(sequences)
        packed_input = pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_output, (hidden, cell) = self.rnn(packed_input)

        if self.bidirectional:
            hidden_forward = hidden[-2]
            hidden_backward = hidden[-1]
            hidden_combined = torch.cat((hidden_forward, hidden_backward), dim=1)
        else:
            hidden_combined = hidden[-1]

        logits = self.output_layer(hidden_combined)
        return logits

# Hiperparámetros
vocab_size = 10  # Dígitos del 0 al 9
embedding_dim = 16
hidden_size = 32
output_dim = 2  # Clases: par e impar
num_layers = 2
bidirectional = True
batch_size = 64
num_epochs = 20
learning_rate = 0.001

# Inicialización del modelo, función de pérdida y optimizador
model = ClasificadorSecuencia(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    output_dim=output_dim,
    num_layers=num_layers,
    bidirectional=bidirectional
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento
for epoch in range(num_epochs):
    model.train()
    sequences, labels = generar_lote(batch_size)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)

    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)

    optimizer.zero_grad()

    logits = model(padded_sequences, lengths)

    loss = criterion(logits, labels)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 1 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Pérdida: {loss.item():.4f}')

# Evaluación
model.eval()

with torch.no_grad():
    test_sequences, test_labels = generar_lote(batch_size)
    lengths = torch.tensor([len(seq) for seq in test_sequences], dtype=torch.long)
    padded_sequences = pad_sequence(test_sequences, batch_first=True, padding_value=0)
    logits = model(padded_sequences, lengths)
    predictions = torch.argmax(logits, dim=1)
    accuracy = (predictions == test_labels).float().mean()
    print(f'Precisión en prueba: {accuracy.item():.4f}')


Época [1/20], Pérdida: 0.6989
Época [2/20], Pérdida: 0.7008
Época [3/20], Pérdida: 0.6834
Época [4/20], Pérdida: 0.6908
Época [5/20], Pérdida: 0.6922
Época [6/20], Pérdida: 0.6888
Época [7/20], Pérdida: 0.6977
Época [8/20], Pérdida: 0.6992
Época [9/20], Pérdida: 0.7038
Época [10/20], Pérdida: 0.6992
Época [11/20], Pérdida: 0.6904
Época [12/20], Pérdida: 0.6927
Época [13/20], Pérdida: 0.6917
Época [14/20], Pérdida: 0.6947
Época [15/20], Pérdida: 0.7001
Época [16/20], Pérdida: 0.6924
Época [17/20], Pérdida: 0.6946
Época [18/20], Pérdida: 0.6975
Época [19/20], Pérdida: 0.6914
Época [20/20], Pérdida: 0.6899
Precisión en prueba: 0.3750


### Detalles adicionales y explicaciones


En este ejemplo, no implementamos explícitamente la compartición de pesos en una red totalmente conectada, ya que PyTorch no soporta directamente esta funcionalidad en capas lineales estándar. Sin embargo, podemos ilustrarlo definiendo una capa personalizada que reutiliza los mismos pesos en múltiples capas.

```python
class CapaCompartida(nn.Module):
    def __init__(self, input_size, output_size):
        super(CapaCompartida, self).__init__()
        self.pesos_compartidos = nn.Linear(input_size, output_size)

    def forward(self, x):
        # Aplicamos la misma capa varias veces
        x = self.pesos_compartidos(x)
        x = torch.relu(x)
        x = self.pesos_compartidos(x)
        x = torch.relu(x)
        return x
```

**Manejo de secuencias con padding y packing**

- **Padding:** Rellenamos las secuencias más cortas con un valor (0 en este caso) para que todas tengan la misma longitud.
- **Packing:** Utilizamos `pack_padded_sequence` para indicar a la RNN las longitudes reales de las secuencias, evitando computaciones innecesarias en los pasos de padding.


La capa de embedding transforma los índices enteros de los dígitos en vectores densos de dimensión `embedding_dim`.

**Uso del último estado oculto para predicciones**

Utilizamos el estado oculto final de la RNN (o la concatenación de los estados ocultos finales en ambas direcciones si es bidireccional) para hacer la predicción.


**RNNs más complejas**

- **Múltiples capas:** Especificado por `num_layers`. Cada capa toma la salida de la capa anterior como su entrada.
- **Bidireccionalidad:** Si `bidirectional=True`, la RNN procesa la secuencia en ambas direcciones (hacia adelante y hacia atrás).

**Entradas empaquetadas y desempaquetadas simultáneamente**

Si necesitamos acceder a las salidas en cada paso temporal (por ejemplo, para tareas de secuencia a secuencia), podemos desempaquetar las secuencias después de la RNN.

```python
# Después de la RNN
packed_output, (hidden, cell) = self.rnn(packed_input)

# Desempaquetamos las secuencias
output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

# Ahora output tiene forma [batch_size, max_length, hidden_size * num_directions]
```


### Ejercicios

**Compartición de pesos en una red totalmente conectada**

**Ejercicio:**

Implementa en PyTorch una red neuronal totalmente conectada con múltiples capas ocultas que comparten el mismo conjunto de pesos y sesgos. La red debe:

- Tener una capa de entrada, tres capas ocultas y una capa de salida.
- Utilizar la función de activación ReLU en las capas ocultas.
- Predecir la clase de entrada en una tarea de clasificación simple (por ejemplo, clasificación de dígitos escritos a mano).

**Puntos a abordar:**

- Define una clase de PyTorch `nn.Module` que reutilice los mismos pesos en todas las capas ocultas.
- Explica cómo la compartición de pesos afecta el número total de parámetros y el entrenamiento del modelo.
- Entrena y evalúa el modelo en un conjunto de datos, como MNIST.

**Pista:**

- Utiliza un solo `nn.Linear` para las capas ocultas y aplícalo varias veces en el método `forward`.
  
---

**2. Compartición de pesos en el tiempo**

**Ejercicio:**

Implementa manualmente una red neuronal recurrente simple (RNN) sin utilizar las clases predefinidas de PyTorch como `nn.RNN`, `nn.LSTM` o `nn.GRU`. Tu implementación debe:

- Procesar una secuencia de números (por ejemplo, una serie temporal).
- Compartir los mismos pesos recurrentes a lo largo del tiempo.
- Predecir el siguiente valor en la secuencia (tarea de predicción secuencial).

**Puntos a abordar:**

- Define los pesos de entrada y recurrentes manualmente.
- Escribe el bucle temporal en el método `forward` de tu clase `nn.Module`.
- Demuestra cómo se comparten los pesos a lo largo del tiempo.

**Pista:**

- Utiliza una función de activación como `torch.tanh` o `torch.relu` en cada paso temporal.
  
---

**3. Clasificación de secuencias en PyTorch**

**Ejercicio:**

Desarrolla un modelo en PyTorch para clasificar secuencias de texto como positivas o negativas (análisis de sentimiento). El modelo debe:

- Utilizar una capa de embedding para representar las palabras.
- Emplear una RNN (LSTM o GRU) para procesar las secuencias.
- Hacer predicciones basadas en el último estado oculto.

**Puntos a abordar:**

- Preprocesa el texto (tokenización, creación de vocabulario, secuenciación).
- Maneja secuencias de diferentes longitudes con padding y packing.
- Entrena y evalúa el modelo en un conjunto de datos, como IMDb.

**Pista:**

- Puedes utilizar `torchtext` para facilitar el manejo de datos de texto.
  
---

**4. Implementación de capas de embedding**

**Ejercicio:**

Implementa una capa de embedding personalizada que:

- Inicialice los embeddings con valores aleatorios o preentrenados (por ejemplo, Word2Vec).
- Permita congelar (no entrenar) o descongelar (entrenar) los embeddings durante el entrenamiento.
- Sea compatible con el uso de padding y packing.

**Puntos a abordar:**

- Explica cómo afecta al modelo el hecho de entrenar o no los embeddings.
- Prueba ambas opciones y compara los resultados en términos de precisión y tiempo de entrenamiento.

**Pista:**

- Controla el parámetro `requires_grad` de los embeddings para congelarlos o descongelarlos.
  
---

**5. Predicciones usando el último paso temporal**

**Ejercicio:**

Modifica el modelo de clasificación de secuencias para que, en lugar de usar solo el último estado oculto, combine los estados ocultos de todos los pasos temporales (por ejemplo, mediante una media o una suma ponderada). Evalúa cómo afecta esta modificación al rendimiento del modelo.

**Puntos a abordar:**

- Implementa una forma de agregar información de todos los estados ocultos.
- Compara los resultados con el modelo que utiliza solo el último estado oculto.
- Discute las ventajas y desventajas de cada enfoque.

**Pista:**

- Puedes utilizar `torch.mean` o `torch.sum` a lo largo de la dimensión temporal.
  
---

**Mejora del tiempo de entrenamiento con packing**

**Ejercicio:**

Entrena una RNN en un conjunto de datos con secuencias de diferentes longitudes. Mide y compara el tiempo de entrenamiento y el rendimiento del modelo al usar:

- Entrenamiento sin padding ni packing (procesamiento individual de secuencias).
- Entrenamiento con padding pero sin packing.
- Entrenamiento con padding y packing.

**Puntos a abordar:**

- Implementa el manejo de secuencias con padding y packing.
- Explica cómo el packing mejora la eficiencia del entrenamiento.
- Presenta los tiempos de entrenamiento y las métricas de rendimiento en cada caso.

**Pista:**

- Utiliza `time.time()` o la función `perf_counter` para medir el tiempo de entrenamiento.
  
---

**7. Entrenamiento de una RNN en lotes**

**Ejercicio:**

Crea una RNN que pueda entrenarse en lotes con secuencias de diferentes longitudes para predecir la siguiente palabra en una secuencia de texto (modelo de lenguaje). Asegúrate de:

- Manejar correctamente el padding y packing durante el entrenamiento en lotes.
- Actualizar los estados ocultos entre lotes de forma adecuada.
- Evaluar el modelo en términos de pérdida de entropía cruzada y precisión de predicción.

**Puntos a abordar:**

- Preprocesa el texto para crear un conjunto de datos adecuado.
- Implementa un `DataLoader` personalizado si es necesario.
- Discute cómo se maneja el estado oculto en el entrenamiento en lotes.

**Pista:**

- Al final de cada lote, reinicia el estado oculto o utiliza técnicas para mantener la continuidad si es necesario.
  
---

**8. Entradas empaquetadas y desempaquetadas simultáneamente**

**Ejercicio:**

Implementa un modelo seq2seq (secuencia a secuencia) para traducción automática que:

- Utilice una RNN como codificador (encoder) y otra como decodificador (decoder).
- Maneje correctamente las secuencias empaquetadas y desempaquetadas en el codificador y decodificador.
- Aplique un mecanismo de atención para mejorar la calidad de la traducción.

**Puntos a abordar:**

- Asegúrate de que el codificador procese secuencias empaquetadas.
- Desempaqueta las salidas del codificador si es necesario para el decodificador.
- Implementa y explica el mecanismo de atención utilizado.

**Pista:**

- El decodificador generalmente procesa un paso a la vez, por lo que puede no requerir empaquetamiento.
  
---

**9. RNNs más Complejas: múltiples capas**

**Ejercicio:**

Construye una RNN profunda con múltiples capas ocultas para realizar reconocimiento de voz básico. El modelo debe:

- Procesar espectrogramas o características de audio (como MFCCs).
- Utilizar una RNN con al menos tres capas ocultas.
- Manejar secuencias de entrada de longitud variable.

**Puntos a abordar:**

- Preprocesa los datos de audio para extraer características relevantes.
- Explica cómo las múltiples capas permiten capturar características más complejas.
- Evalúa el modelo en términos de precisión y tasa de error.

**Pista:**

- Puedes utilizar conjuntos de datos como el Librispeech o el dataset de comandos de voz de Google.
  
---

**RNNs bidireccionales**

**Ejercicio:**

Implementa una RNN bidireccional para realizar etiquetado de secuencias en texto (por ejemplo, reconocimiento de entidades nombradas o etiquetado POS). Compara su rendimiento con una RNN unidireccional y analiza las diferencias.

**Puntos a abordar:**

- Explica cómo la bidireccionalidad ayuda a capturar dependencias contextuales.
- Implementa ambas versiones del modelo y entrena en un conjunto de datos etiquetado.
- Presenta y compara los resultados obtenidos.

**Pista:**

- Utiliza conjuntos de datos etiquetados como CoNLL-2003 o Penn Treebank.
  
---

**Desafío adicional: implementación de un modelo de lenguaje con LSTM**

**Ejercicio:**

Crea un modelo de lenguaje utilizando una LSTM en PyTorch que sea capaz de generar texto similar a una fuente dada (por ejemplo, obras de Shakespeare). El modelo debe:

- Procesar secuencias de texto y predecir la siguiente palabra o carácter.
- Manejar eficientemente el estado oculto entre secuencias.
- Generar texto nuevo a partir de una semilla inicial.

**Puntos a abordar:**

- Preprocesa el texto para crear un vocabulario y secuencias de entrenamiento.
- Implementa el entrenamiento del modelo y el proceso de generación de texto.
- Ajusta los hiperparámetros para mejorar la calidad del texto generado.

**Pista:**

- Experimenta con diferentes tamaños de secuencia y dimensiones del estado oculto.
  
---

####  **11. Atención en RNNs**

**Ejercicio:**

Añade un mecanismo de atención a un modelo seq2seq de traducción automática. Tu modelo debe:

- Utilizar una RNN para el codificador y otra para el decodificador.
- Implementar atención según los modelos de Bahdanau o Luong.
- Mostrar cómo la atención mejora el rendimiento del modelo.

**Puntos a abordar:**

- Explica el concepto de atención y su importancia en modelos seq2seq.
- Implementa el cálculo de los pesos de atención y cómo se aplican al decodificador.
- Evalúa el modelo con y sin atención y compara los resultados.

**Pista:**

- Visualiza los pesos de atención para interpretar cómo el modelo se enfoca en diferentes partes de la entrada.
  

In [ ]:
## Tus respuestas